In [ ]:
import matplotlib.dates as mdates
import mpl_panel_builder as mpb
import numpy as np
import pandas as pd
from mpl_panel_builder.helpers.mpl import adjust_axes_size

from loadshift import moving_horizon

# Use autoreload magic so that .py files can modified without having to
# restart the kernel
%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_csv("../data/example_data.csv", index_col=0, parse_dates=True)
# Concatenate data to create a 3-day long dataset
data_conc = pd.concat([data] * 3, ignore_index=True)
data_conc.index = pd.date_range(
    start=data.index[0],
    periods=len(data_conc),
    freq="h"
)

mpb_config = {
    "panel": {
        "dimensions": {"width_cm": 20, "height_cm": 6},
        "margins": {
            "left_cm": 0.75,
            "bottom_cm": 1.25,
            "right_cm": 0.25,
            "top_cm": 0,
        },
        "axes_separation": {"x_cm": 0.5, "y_cm": 0.75},
    },
    "style": {
        "theme": "presentation",
        "rc_params": {
            "figure.facecolor": "white",
            "axes.facecolor": "none",
            "legend.facecolor": "white",
            "font.size": 10,
        },
    },
    "output": {
        "format": "png",
        "dpi": "600"
    },
}

In [ ]:
red = "#EB6D44"
gray = "#9EADB2"
green = "#50AA46"
bar_width = 1/24*0.8
max_rate = 0.5
flexibility = 2
xlim = [
    data_conc.index[0] - pd.Timedelta(hours=1),
    data_conc.index[-1] + pd.Timedelta(hours=1),
]

config = {
    "daily_decision_hour": 0,
    "n_lookahead_hours": 36,
    "load_shift": {
        "max_demand_advance": flexibility,
        "max_demand_delay": flexibility,
        "max_hourly_purchase": 2*max_rate,
        "max_rate": max_rate
    }
}

mpb.configure(mpb_config)
mpb.set_rc_style()
fig, axs = mpb.create_panel(rows=2)

ax = axs[0][0]
adjust_axes_size(ax, length_cm=1, direction="top")
ax.plot(data_conc.index, data_conc["price"], 'k-',solid_capstyle='round')
ax.set(xlim=xlim, xticks=[], yticks=[], title="Moving horizon example")
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_ylabel("Price")



optimizer = moving_horizon(
    price_data=data_conc[["price"]], 
    demand_data=data_conc[["demand"]], 
    config=config
)
shift = optimizer["results"]["shift"]

remove = np.minimum(shift, 0)
add = np.maximum(shift, 0)

ax = axs[1][0]
ax.bar(
    data_conc.index,
    data_conc["demand"],
    bar_width,
    fc=gray,
    label="Original demand",
)
ax.bar(
    data_conc.index,
    add,
    bar_width,
    bottom=data_conc["demand"],
    fc=green,
    label="Added demand",
)
ax.bar(
    data_conc.index,
    -remove,
    bar_width,
    bottom=data_conc["demand"]+remove,
    fc=red,
    label="Removed demand",
)

ax.set(xlabel="Time", xlim=xlim, yticks=[], ylim=[0, 2*max_rate], ylabel="Demand")
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0, 12]))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_linewidth(3)
ax.spines['bottom'].set_capstyle('round')

ax.legend(
    loc="center",
    bbox_to_anchor=(0.5, 1),
    ncol=3
    )

mpb.save_panel(fig, "moving_horizon_example")